In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import csv

In [ ]:
def scrap_malt(n):
    all_links = []  # Créez une liste pour stocker tous les liens

    for i in range(n):
        # Remplacez 'chemin_vers_le_driver' par le chemin complet vers votre pilote de navigateur.
        driver = webdriver.Firefox()
        wait = WebDriverWait(driver, 2)
        # Parcourir les pages web du site
        url = "https://www.malt.fr/all-freelances?p=" + str(i+1)

        # Ouvrir la page web
        driver.get(url)

        # Attendre que la section se charge (vous pouvez ajuster cela en fonction de vos besoins)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "profile-card")))

        # Obtenir le code source de la page
        page_source = driver.page_source

        # Utilisez BeautifulSoup pour analyser le HTML
        soup = BeautifulSoup(page_source, 'html.parser')

        # Trouver le lien dans la section
        links = soup.find_all('section', class_='profile-card')  # Trouver tous les liens sur la page

        # Parcourir les liens et les ajouter à la liste
        for link in links:
            data_target = link['data-target']
            full_link = "https://www.malt.fr" + data_target
            all_links.append(full_link)
        # Fermer le navigateur
        driver.quit()

    # Retourner la liste des liens récupérés
    return all_links

In [ ]:
links = scrap_malt(1) # Vous pouvez spécifier le nombre de pages que vous souhaitez extraire
for link in links:
    print(link)

In [ ]:
def scrap_malt_user(links):
    all_data = []

    for link in links:
        driver = webdriver.Firefox()
        wait = WebDriverWait(driver, 10)
        driver.get(link)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        
        data = {}

        price_element = soup.find('div', {'data-testid': 'profile-price'})
        if price_element:
            price = price_element.find('span', class_='block-list__price').text
            data['price'] = price.strip()
        
        # Récupérer l'expérience
        experience_element = soup.find('span', text='Expérience')
        if experience_element:
            experience = experience_element.find_next('span', class_='profile-indicators-content').text
            data['experience'] = experience.strip()
        
        # Récupérer le taux de réponse
        response_rate_element = soup.find('span', text='Taux de réponse')
        if response_rate_element:
            response_rate = response_rate_element.find_next('span', class_='profile-indicators-content').text
            data['response_rate'] = response_rate.strip()
        
        # Récupérer le temps de réponse
        response_time_element = soup.find('span', text='Temps de réponse')
        if response_time_element:
            response_time = response_time_element.find_next('span', class_='profile-indicators-content').text
            data['response_time'] = response_time.strip()
            
        # Récupérer le nom 
        name_element = soup.find('div', {'data-testid': 'profile-fullname'})
        if name_element:
            name = name_element.text
            data['name'] = name.strip()
            
        # Récupérer le métier
        headline_element = soup.find('div', {'data-testid': 'profile-headline'})
        if headline_element:
            headline = headline_element.text
            data['headline'] = headline.strip()
            
        # Récupérer le nombre de missions
        missions_element = soup.find('div', {'data-testid': 'profile-counter-missions'})
        if missions_element:
            missions = missions_element.find('strong').text
            data['missions'] = missions.strip()
            
        # Récupérer toutes les catégories
        categories_elements = soup.find_all('li', {'class': 'categories__list-item'})
        categories = [category.find('a').text for category in categories_elements]
        data['categories'] = categories
        
        
        # Récupérer les compétences
        competences_element = soup.find_all('div', {'class': 'profile-expertises__content-list-item__label'})
        competences = [competence.find('a', class_='joy-link joy-link_teal').text.strip() for competence in competences_element]

        data['competences'] = competences
        
        # Récupérer le statut "Supermalter"
        supermalter_element = soup.find('span', class_='joy-badge-level__tag blue')
        if supermalter_element:
            supermalter = supermalter_element.get_text(strip=True)
            data['supermalter'] = supermalter
            
        # Récupérer la localisation
        location_element = soup.find('dl', {'class': 'profile__location-and-workplace-preferences__item'})
        if location_element:
            location_label = location_element.find('dt', {'data-testid': 'profile-location-address-label'})
            location_value = location_element.find('dd', {'data-testid': 'profile-location-preference-address'})

            if location_label and location_value:
                location = {location_label.text: location_value.text}
                data['location'] = location
                
        # Récupérer la préférence de télétravail
        teletravail_element = soup.find('dl', {'class': 'profile-page-mission-preferences__item'})
        if teletravail_element:
            teletravail_label = teletravail_element.find('dt')
            teletravail_value = teletravail_element.find('dd')

            if teletravail_label and teletravail_value:
                teletravail_preference = {teletravail_label.text: teletravail_value.text}
                data['teletravail_preference'] = teletravail_preference
                
        # Récupérer le nombre de recommandations
        recommendations_element = soup.find('span', {'data-testid': 'profile-counter-recommendations'})
        if recommendations_element:
            recommendations_count = int(recommendations_element.text.split()[0])
            data['recommendations'] = recommendations_count   
            

        # Récupérer le message de présentation
        presentation_element = soup.find('div', {'class': 'profile-description__content'})
        if presentation_element:
            presentation_message = presentation_element.get_text(strip=True)
            data['presentation'] = presentation_message
            
            
        all_data.append(data)
        
        driver.quit()

    return all_data

In [20]:
liens = links
data = scrap_malt_user(liens[:3])

In [21]:
data

[{'price': '260\xa0€',
  'experience': '8-15 ans',
  'response_rate': '100%',
  'response_time': '1h',
  'name': 'Aurélie Poupée',
  'headline': 'Concepteur Rédacteur / Copywriter / Rédacteur web',
  'missions': '1\u202f085',
  'categories': ['Concepteur Rédacteur', 'Pigiste', 'Rédacteur web'],
  'competences': ['Conception rédaction',
   'Copywriting',
   'Rédaction web SEO',
   'Livre Blanc',
   'Site vitrine',
   'Rédaction',
   'Interview',
   'Articles',
   'Rédaction web',
   'Copywriter',
   'Article de blog',
   'Stratégie éditoriale',
   'Responsable éditorial',
   'Cas client',
   'Rédaction SEO',
   'Consulting éditorial',
   'UX Writing',
   'Blog',
   'Contenu éditorial',
   'Storytelling',
   'Ebook',
   'Réécriture',
   'Pilotage de projets',
   'rewriting',
   'Brand content',
   'Inbound marketing',
   'Landing page',
   'Rédaction de contenu',
   'Marketing de contenu',
   'Tone of voice',
   'Charte éditoriale',
   'Journalisme',
   'documentaliste',
   'Relecture',


In [22]:
def save_to_csv(data, csv_filename):
    # Assurez-vous que la liste de données n'est pas vide
    if not data:
        print("Aucune donnée à enregistrer.")
        return

    # Créez une liste de noms de colonnes pour le CSV en incluant les nouveaux éléments
    fieldnames = ['name', 'headline', 'price', 'experience', 'response_rate', 'response_time', 'missions', 'categories', 'competences', 'supermalter', 'location','presentation', 'recommendations', 'teletravail_preference']

    # Ouvrir le fichier CSV en mode écriture
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

        # Écrire les en-têtes
        writer.writeheader()

        # Écrire les données
        for entry in data:
            writer.writerow(entry)

    print(f"Données enregistrées dans {csv_filename}")

In [23]:
csv_filename = 'malt_data.csv'
save_to_csv(data, csv_filename)

Données enregistrées dans malt_data.csv


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

driver = webdriver.Firefox()
wait = WebDriverWait(driver, 10)
driver.maximize_window()
driver.get("https://discpersonalitytesting.com/login-here/")
email = "myemail@email.com"
password = "mypassword1@23"
email_textfield = driver.find_element_by_name("email")
password_textfield = driver.find_element_by_name("password")
login_button = driver.find_element_by_name("LoginDAPLoginForm")
email_textfield.send_keys(email)
time.sleep(2)
password_textfield.send_keys(password)
time.sleep(5)
login_button.click()